## Translation Pipeline

This notebook is a summary of the code explained in a series or [articles](https://medium.com/@dbenzaquenm) that aim to introduce recurrent neural networks. More precisely to the article titled *How to build a translation pipeline with RNN and Keras*.


#### Imports

In [ ]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, Bidirectional
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
# Path to translation file
path_to_data = 'data/spa.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs = pairs[:-1] # skip last empty element

In [ ]:
pairs = pairs[1000:20000]

for idx_sample in range(5,10):
    print('English example in pair {}:  {}'.format(idx_sample + 1, pairs[idx_sample][0]))
    print('Spanish example in pair {}:  {}'.format(idx_sample + 1, pairs[idx_sample][1]))

#### Clean Data

In [ ]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

print(clean_sentence("I will surf today !!"))

#### Tokenize

In [ ]:
text_examples = [
    'i will surf today',
    'this week i will travel to the beach',
    'he went to his house by the beach',]

# Create tokenizer
exp_text_tokenizer = Tokenizer()
# Create word index
exp_text_tokenizer.fit_on_texts(text_examples)
for key, value in exp_text_tokenizer.word_index.items():
    print("Word: {} is converted to number {}".format(key, value))
    
    
# Tokenize sentences
exp_text_tokenized = exp_text_tokenizer.texts_to_sequences(text_examples)
print ('\n')
for sample_i, (sent, token_sent) in enumerate(zip(text_examples, exp_text_tokenized)):
    print('Input sentence:  {}'.format(sent))
    print('Output vector: {} \n'.format(token_sent))

In [ ]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:
# Clean sentences
english_sentences = [clean_sentence(pair[0]) for pair in pairs]
spanish_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
spa_text_tokenized, spa_text_tokenizer = tokenize(spanish_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length spanish sentence: {}'.format(len(max(spa_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))

# Check language length
spanish_vocab = len(spa_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of {} unique words".format(spanish_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

#### Padding

In [ ]:
text_examples = ['i will surf today',
    'this week i will travel to the beach',
    'he went to his house by the beach',]

In [ ]:
print('Maximum length of example sentence: {}'.format(len(max(exp_text_tokenized,key=len))))
# Pad tokenize vectors
exp_pad_sentence = pad_sequences(exp_text_tokenized, 8, padding = "post") # 8 is the max length
for index, pad_sentence in enumerate(exp_pad_sentence):
    print("Example sentence {}:".format(index+1))
    print("  -Input:{}".format(exp_text_tokenized[index]))
    print("  -Output:{}".format(pad_sentence))

In [ ]:
max_sentence_length = 12
spa_pad_sentence = pad_sequences(spa_text_tokenized, max_sentence_length, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_sentence_length, padding = "post")

# Reshape data
spa_pad_sentence = spa_pad_sentence.reshape(*spa_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

### Create simple RNN Model

<div class="alert alert-block alert-info">
<b>Warning:</b> Tha paremeters of the model are not optimized</div>

In [ ]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
input_shape = (max_sentence_length, 1)

In [ ]:
input_shape = (max_sentence_length, 1)
input_sequence = Input(input_shape, name='InputLayer')
rnn = LSTM(256, return_sequences=True, dropout=0.5, name='RNNLayer')(input_sequence)
logits = TimeDistributed(Dense(spanish_vocab), name='TimeDistributed')(rnn)

model = Model(input_sequence, Activation('softmax')(logits))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-2),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model_results = model.fit(eng_pad_sentence, spa_pad_sentence, batch_size=30, epochs=1)

In [ ]:
index = 10
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(spanish_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(model.predict(eng_pad_sentence[index:index+1])[0], spa_text_tokenizer))


If you have enjoyed this first tutorial you can find improved models in the following link

You can also find theoretical explanation in my [Medium profile](https://medium.com/@dbenzaquenm) 